In [2]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import env
import acquire as a
from os.path import isfile
from importlib import reload

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

In [3]:
# def acquire_cohorts()->pd.DataFrame:
#     if isfile('cohort_logs.csv'):
#         return pd.read_csv('cohort_logs.csv',index_col=[0])
#     query = '''SELECT id AS cohort, name AS cohort_name , start_date, end_date, program_id FROM cohorts'''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     df = pd.read_sql(query,url)
#     df.index = df.cohort
#     return cohorts_df

In [6]:
# def acquire_curriculum_data():
#     '''This function gets curriculum data from the local repo, which is stored
#     as a .txt file'''
#     #Get the data from text file on local repo
#     df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
#     df.date = pd.to_datetime(df.date + ' ' + df.time)
#     df = df.drop(columns=['time'])
#     df = df.set_index(df['date'])
#     df = df.drop(columns=['date'])               
#     return df

In [10]:
# def clean_cohort_logs(df):
#     #Get csv from file
#     df = pd.read_csv('cohort_logs.csv', index_col=[0])
#     #Changing date, start and end dates to datetime fields
#     df["date"]= pd.to_datetime(df["date"])
#     df["start_date"]= pd.to_datetime(df["start_date"])
#     df["end_date"]= pd.to_datetime(df["end_date"])
#     #Setting index as date time
#     #Set datetime index
#     df = df.set_index(df.date)
#     #Creating new column for program length
#     df['program_length'] = df.end_date - df.start_date
#     #Create column with program name
#     #Change data types as needed
#     df["program_id"]= df["program_id"].astype(str)
#     #Copying these values to a new column program name
#     df['program_name'] = df['program_id']
#     #Mapping the real names of the program
#     df["program_name"] = df["program_name"].map({'1.0':'full stack PHP','2.0':'full stack Java','3.0':'data science','4.0':'front end'})
    
#     return df

In [11]:
# def wrangle_curriculum_access()->pd.DataFrame:
#     curriculum_df = acquire_curriculum_data()
#     cohort_df = acquire_cohorts()
#     cohort_df = clean_cohort_logs(cohort_df)
#     curriculum_df = curriculum_df.join(cohort_df,on='cohort',how='left',lsuffix='_id')
#     curriculum_df = curriculum_df.drop(columns=['cohort_id','cohort','program_id'])
#     curriculum_df.program_name = curriculum_df.program_name.astype('category')
#     return curriculum_df

In [4]:
reload(a)
curriculum_df = a.wrangle_curriculum_access()
curriculum_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java


In [13]:
curriculum_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            900222 non-null  object         
 1   id              900223 non-null  int64          
 2   ip              900223 non-null  object         
 3   cohort_name     847330 non-null  object         
 4   start_date      847330 non-null  datetime64[ns] 
 5   end_date        847330 non-null  datetime64[ns] 
 6   program_length  847330 non-null  timedelta64[ns]
 7   program_name    847330 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 55.8+ MB


In [2]:
# def acquire_cohorts():
#     if isfile('cohorts.csv'):
#         return pd.read_csv('cohorts.csv',index_col=[0])
#     query = '''SELECT id AS cohort, name AS cohort_name , start_date, end_date, program_id FROM cohorts'''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     df = pd.read_sql(query,url)
#     df.index = df.cohort
#     return df

In [3]:
# df = acquire_cohorts()

NameError: name 'isfile' is not defined

In [2]:
# # Import .txt file and convert it to a DataFrame object
# df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [3]:
# def acquire_curriculum_data():
#     #Get the data from text file on local repo
#     df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
#                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])
#     return df

In [4]:
# df = acquire_curriculum_data()

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [5]:
#Total Users
curriculum_df.id.nunique()

981

In [6]:
curriculum_df.id.value_counts().tail(20)

348    3
176    3
177    3
852    2
97     2
573    2
574    2
649    1
940    1
165    1
879    1
652    1
592    1
619    1
593    1
66     1
163    1
918    1
212    1
952    1
Name: id, dtype: int64

In [20]:
curriculum_df.ip.nunique()

5531

In [25]:
curriculum_df.ip.value_counts()

97.105.19.58       284579
97.105.19.61        61662
192.171.117.210      9515
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.58.109.140          1
99.203.213.59           1
99.203.212.173          1
173.239.198.246         1
72.181.124.244          1
Name: ip, Length: 5531, dtype: int64

In [29]:
curriculum_df.cohort.value_counts()

28.0     84031
33.0     40730
29.0     38096
62.0     37109
53.0     36902
24.0     35636
57.0     33844
56.0     33568
51.0     32888
59.0     32015
22.0     30926
58.0     29855
32.0     29356
23.0     28534
52.0     28033
26.0     27749
34.0     26538
25.0     25586
31.0     25359
132.0    23691
55.0     21582
27.0     20743
61.0     17713
134.0    16623
135.0    16397
133.0    14715
14.0      9587
1.0       8890
137.0     8562
21.0      7444
138.0     7276
17.0      4954
13.0      2845
18.0      2158
8.0       1712
139.0     1672
19.0      1237
16.0       755
15.0       691
7.0        598
12.0       302
11.0       253
2.0         93
6.0         72
9.0          5
4.0          4
5.0          1
Name: cohort, dtype: int64

In [32]:
curriculum_df.isnull().sum()

date          0
time          0
page          1
id            0
cohort    52893
ip            0
dtype: int64

In [8]:
curriculum_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            900222 non-null  object         
 1   id              900223 non-null  int64          
 2   ip              900223 non-null  object         
 3   cohort_name     847330 non-null  object         
 4   start_date      847330 non-null  datetime64[ns] 
 5   end_date        847330 non-null  datetime64[ns] 
 6   program_length  847330 non-null  timedelta64[ns]
 7   program_name    847330 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 55.8+ MB


In [6]:
# def acquire_cohort_logs(user=env.user, password=env.password, host=env.host):
#     '''
#     This function queries the Codeup MySQL curriculum_logs database and returns a dataframe
#     '''
#     url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
#     query = '''
#     SELECT date, path as endpoint, user_id, cohort_id, name, start_date, end_date, ip as source_ip, program_id
#         FROM logs l
#         LEFT JOIN cohorts c ON l.cohort_id = c.id;
#     '''
#     df = pd.read_sql(query, url)
#     #Save file to csv
#     df.to_csv('cohort_logs.csv')
#     return df

In [9]:
#Taking all activity for students, not staff
student_df = curriculum_df[curriculum_df.cohort_name != "Staff"]
student_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,2015-09-22,2016-02-06,137 days,full stack PHP
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java


In [11]:
#Taking all activity from students active in course(from start date to end date)
active_df = student_df[(student_df.end_date >= student_df.index) & (student_df.start_date <= student_df.index)]
active_df = active_df[active_df['page'] != '/']
active_df.head()

,page,id,ip,cohort_name,start_date,end_date,program_length,program_name
date,,,,,,,,
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-26 09:56:41,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-26 09:56:46,javascript-i/conditionals,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-26 09:56:48,javascript-i/functions,3,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java
2018-01-26 09:56:59,javascript-i/loops,2,97.105.19.61,Teddy,2018-01-08,2018-05-17,129 days,full stack Java


In [12]:
#617k active student log entries
active_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 616912 entries, 2018-01-26 09:56:24 to 2021-04-21 16:41:51
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   page            616911 non-null  object         
 1   id              616912 non-null  int64          
 2   ip              616912 non-null  object         
 3   cohort_name     616912 non-null  object         
 4   start_date      616912 non-null  datetime64[ns] 
 5   end_date        616912 non-null  datetime64[ns] 
 6   program_length  616912 non-null  timedelta64[ns]
 7   program_name    616912 non-null  category       
dtypes: category(1), datetime64[ns](2), int64(1), object(3), timedelta64[ns](1)
memory usage: 38.2+ MB


In [13]:
#Aggregating pages by student
pages_by_student = active_df.groupby(['user_id'])['endpoint'].value_counts()
pages_by_student

user_id  endpoint                                        
2        java-iii                                            50
         mysql                                               32
         java-ii                                             30
         spring                                              29
         jquery                                              28
                                                             ..
981      examples/css/bootstrap-grids.html                    3
         appendix/professional-development/t-block-resume     1
         examples/bootstrap-grid/assets.zip                   1
         html-css/css-ii                                      1
         javascript-ii                                        1
Name: endpoint, Length: 88202, dtype: int64

In [14]:
#make a df
pages_by_student =pd.DataFrame(pages_by_student)
pages_by_student.head()

endpoint
user_id endpoint          
2       java-iii        50
        mysql           32
        java-ii         30
        spring          29
        jquery          28

In [15]:
pages_by_student.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 88202 entries, (2, 'java-iii') to (981, 'javascript-ii')
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   endpoint  88202 non-null  int64
dtypes: int64(1)
memory usage: 1.0+ MB


In [20]:
pages_by_student.columns = ['endpoint_count']
pages_by_student = pages_by_student.reset_index()
pages_by_student = pages_by_student.groupby('user_id').sum('endpoint_count')
pages_by_student.nsmallest(n=5, columns='endpoint_count')

,endpoint_count
user_id,
832,2
278,3
539,4
956,4
388,7


In [32]:
#Looking at 832, Very little activity, in the middle of the course too, full stack Java Jupiter
user_832 = active_df[active_df.user_id == 832]
user_832.endpoint.value_counts()

javascript-i    1
html-css        1
Name: endpoint, dtype: int64

In [33]:
#Looking at 278, More Activity, more diverse page views, over a larger period of time
user_278 = active_df[active_df.user_id == 278]
user_278.endpoint.value_counts()

java-ii/arrays         2
java-ii/collections    1
Name: endpoint, dtype: int64

In [34]:
user_278

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
date,,,,,,,,,,,
2018-09-27,2018-09-27,java-ii/arrays,278,24.0,Voyageurs,2018-05-29,2018-10-11,107.77.217.9,2.0,135 days,full stack Java
2018-09-27,2018-09-27,java-ii/arrays,278,24.0,Voyageurs,2018-05-29,2018-10-11,107.77.217.9,2.0,135 days,full stack Java
2018-09-27,2018-09-27,java-ii/collections,278,24.0,Voyageurs,2018-05-29,2018-10-11,107.77.217.9,2.0,135 days,full stack Java


In [35]:
#Looking at 539, looking at 
user_539 = active_df[active_df.user_id == 539]
user_539.endpoint.value_counts()

toc                      1
html-css                 1
html-css/introduction    1
html-css/elements        1
Name: endpoint, dtype: int64

In [36]:
user_539

,date,endpoint,user_id,cohort_id,name,start_date,end_date,source_ip,program_id,program_length,program_name
date,,,,,,,,,,,
2019-11-04,2019-11-04,toc,539,52.0,Europa,2019-11-04,2020-04-17,97.105.19.58,2.0,165 days,full stack Java
2019-11-04,2019-11-04,html-css,539,52.0,Europa,2019-11-04,2020-04-17,97.105.19.58,2.0,165 days,full stack Java
2019-11-04,2019-11-04,html-css/introduction,539,52.0,Europa,2019-11-04,2020-04-17,97.105.19.58,2.0,165 days,full stack Java
2019-11-04,2019-11-04,html-css/elements,539,52.0,Europa,2019-11-04,2020-04-17,97.105.19.58,2.0,165 days,full stack Java
